### Par: HOUSNI Kamar

## Projet : Metaheuristic

### Constructive heuristic- Finite Best Strip

* We will start by reading our data;
    * we extract the bin and item dimensions in order to perform our heuristic algorithm on the data.

In [2]:
def read_dataset(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    # first we extract the bin dimensions
    width_bins = int(lines[1].split(':')[1].strip())
    height_bins = int(lines[2].split(':')[1].strip())
    
    # then we extract the item dimensions
    items = []
    for line in line[4:]:
        if line.strip() == '':
            continue
        item_width = int(line.split(':')[1].strip())
        item_height = int(line.split(':')[2].strip())
        items.append((item_width, item_height))
        
        return width_bins, height_bins, items